### Exercise 03 : Aggregations

In [21]:
import pandas as pd
import sqlite3

* create a connection to the database using the library sqlite3
* get the schema of the table test
* get only the first 10 rows of the table test to check what the table looks like

In [22]:
conn = sqlite3.connect('../data/checking-logs.sqlite')
cursor = conn.cursor()

In [23]:
schema = pd.read_sql("PRAGMA table_info(test);", conn)

In [24]:
print(schema)

   cid             name       type  notnull dflt_value  pk
0    0              uid       TEXT        0       None   0
1    1          labname       TEXT        0       None   0
2    2  first_commit_ts  TIMESTAMP        0       None   0
3    3    first_view_ts  TIMESTAMP        0       None   0


In [25]:
preview = pd.read_sql("SELECT * FROM test LIMIT 10;", conn)
preview

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


* find among all the users the minimum value of the delta between the first commit
of the user and the deadline of the corresponding lab using only one query
  * do this by joining the table with the table deadlines
  * the difference should be displayed in hours
  * do not take the lab ’project1’ into account, it has longer deadlines and will be
  an outlier
  * the value should be stored in the dataframe df_min with the corresponding
  uid

In [26]:
query = """
SELECT 
    t.uid,
    CAST(MIN((julianday(t.first_commit_ts) - julianday(datetime(d.deadlines, 'unixepoch'))) * 24) AS INT) AS min_diff_hours
FROM test t
JOIN deadlines d ON t.labname = d.labs
WHERE t.labname != 'project1'
GROUP BY t.uid
ORDER BY min_diff_hours
LIMIT 1
"""
#преобразование Unix timestamp в дату: datetime(d.deadlines, 'unixepoch')

df_min = pd.read_sql(query, conn)
df_min

,uid,min_diff_hours
0,user_30,-202


* do the same thing, but for the maximum, using only one query, the dataframe name is
df_max

In [27]:
query = """
SELECT 
    t.uid,
    MAX((julianday(t.first_commit_ts) - julianday(datetime(d.deadlines, 'unixepoch'))) * 24) AS max_diff_hours
FROM test t
JOIN deadlines d ON t.labname = d.labs
WHERE t.labname != 'project1'
GROUP BY t.uid
ORDER BY max_diff_hours DESC 
LIMIT 1
"""
#преобразование Unix timestamp в дату: datetime(d.deadlines, 'unixepoch')

df_max = pd.read_sql(query, conn)
df_max

,uid,max_diff_hours
0,user_25,-2.867236


In [28]:
query = """SELECT 
    AVG((julianday(t.first_commit_ts) - julianday(datetime(d.deadlines, 'unixepoch'))) * 24) AS avg_diff_hours
FROM test t
JOIN deadlines d ON t.labname = d.labs
WHERE t.labname != 'project1'"""
df_avg = pd.read_sql(query, conn)
df_avg

,avg_diff_hours
0,-89.687686


* we want to test the hypothesis that the users who visited the newsfeed just a few
times have the lower delta between the first commit and the deadline. To do this,
you need to calculate the correlation coefficient between the number of pageviews
and the difference
  * using only one query, create a table with the columns: uid, avg_diff, pageviews
  * uid is the uids that exist in the test
  * avg_diff is the average delta between the first commit and the lab deadline per user
  * pageviews is the number of Newsfeed visits per user
  * do not take the lab ’project1’ into account
  * store it to the dataframe views_diff
  * use the Pandas method corr() to calculate the correlation coefficient between
the number of pageviews and the difference

In [29]:
query = """SELECT 
    t.uid,
    AVG((julianday(t.first_commit_ts) - julianday(datetime(d.deadlines, 'unixepoch'))) * 24) AS avg_diff,
    COUNT(p.datetime) AS pageviews
FROM test t
JOIN deadlines d ON t.labname = d.labs
LEFT JOIN pageviews p ON t.uid = p.uid
WHERE t.labname != 'project1'
GROUP BY t.uid"""

views_diff = pd.read_sql(query, conn)

views_diff

,uid,avg_diff,pageviews
0,user_1,-65.119644,140
1,user_10,-75.242310,445
2,user_14,-159.568696,429
3,user_17,-62.207513,235
4,user_18,-6.367907,9
5,user_19,-99.440298,64
6,user_21,-96.111041,40
7,user_25,-93.474751,895
8,user_28,-86.793652,745
9,user_3,-105.738041,1585


In [30]:
views_diff = pd.read_sql(query, conn)
correlation = views_diff[['avg_diff', 'pageviews']].corr().iloc[0, 1]

views_diff

,uid,avg_diff,pageviews
0,user_1,-65.119644,140
1,user_10,-75.242310,445
2,user_14,-159.568696,429
3,user_17,-62.207513,235
4,user_18,-6.367907,9
5,user_19,-99.440298,64
6,user_21,-96.111041,40
7,user_25,-93.474751,895
8,user_28,-86.793652,745
9,user_3,-105.738041,1585


In [31]:
correlation

-0.18504199436324814

In [32]:
query = """
SELECT 
    t.uid,
    AVG((julianday(t.first_commit_ts) - julianday(datetime(d.deadlines, 'unixepoch'))) * 24) AS avg_diff,
    COUNT(p.datetime) AS pageviews
FROM test t
JOIN deadlines d ON t.labname = d.labs
LEFT JOIN pageviews p ON t.uid = p.uid
WHERE t.labname != 'project1'
GROUP BY t.uid
"""

views_diff = pd.read_sql(query, conn)

correlation_matrix = views_diff[['avg_diff', 'pageviews']].corr()
print(correlation_matrix)

           avg_diff  pageviews
avg_diff   1.000000  -0.185042
pageviews -0.185042   1.000000


In [33]:
views_diff = pd.read_sql(query, conn)
correlation = views_diff[['avg_diff', 'pageviews']].corr().iloc[0, 1]

correlation

-0.18504199436324814

* close the connection

In [34]:
conn.close()